# Monte Carlo

#### For each simulation path:

1. Merchant Revenue = Σ[Generationₜ × Priceₜ] 
2. Implied $/MWh = Merchant Revenue / Total Generation

In [ ]:
# 1. Fit ARIMA/GARCH models to prices
price_model = ARIMA(historical_prices).fit()
residuals = price_model.resid

# 2. Bootstrap residuals for paths
paths = []
for _ in range(10000):
    random_residuals = np.random.choice(residuals, size=term_length, replace=True)
    simulated_prices = price_model.predict() + random_residuals
    paths.append(simulated_prices)

In [ ]:
# 1. Fit distributions to historical prices
rt_busbar_dist = stats.lognorm.fit(historical_rt_busbar)
da_busbar_dist = stats.lognorm.fit(historical_da_busbar)

# 2. Generate correlated samples
correlation_matrix = calculate_correlation_matrix()
multivariate_normal = stats.multivariate_normal(cov=correlation_matrix)

In [ ]:
def calculate_p75_fixed_price(simulated_paths, generation_paths):
    merchant_revenues = []
    
    for path in range(num_simulations):
        total_revenue = sum(simulated_paths[path][t] * generation_paths[path][t] 
                          for t in range(term_length))
        total_generation = sum(generation_paths[path])
        implied_price = total_revenue / total_generation
        merchant_revenues.append(implied_price)
    
    p75_price = np.percentile(merchant_revenues, 25)  # 75% beat merchant
    return p75_price

# Calculate for each product
p75_rt_busbar = calculate_p75_fixed_price(rt_busbar_paths, generation_paths)
p75_rt_hub = calculate_p75_fixed_price(rt_hub_paths, generation_paths)
p75_da_busbar = calculate_p75_fixed_price(da_busbar_paths, generation_paths) 
p75_da_hub = calculate_p75_fixed_price(da_hub_paths, generation_paths)